<a href="https://colab.research.google.com/github/bansi1008/ndvi/blob/main/SAFE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rasterio numpy matplotlib

print("Libraries installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 63.3 MB/s eta 0:00:00
Libraries installed successfully!


In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

print("Google Drive mounted successfully!")
print("You can now access your 'My Drive' contents at /content/drive/My Drive/")

Mounted at /content/drive
Google Drive mounted successfully!
You can now access your 'My Drive' contents at /content/drive/My Drive/


In [ ]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from rasterio.plot import show


download_directory = "/content/drive/My Drive"

print(f"Set download directory to: {download_directory}")

# --- Helper function to find the band files ---
def find_band_file(base_dir, band_name_part):

    for file in os.listdir(base_dir):
        if band_name_part in file and (file.endswith('.jp2') or file.endswith('.tif')):
            return os.path.join(base_dir, file)
    return None

print("Searching for band files...")


red_band_path = find_band_file(download_directory, 'B04_10m')
nir_band_path = find_band_file(download_directory, 'B08_10m')


if red_band_path and nir_band_path:
    print(f"Found Red band (B04): {red_band_path}")
    print(f"Found NIR band (B08): {nir_band_path}")


    try:
        # Open Red band and read its data and profile (metadata)
        with rasterio.open(red_band_path) as src_red:
            red = src_red.read(1).astype(float)
            profile = src_red.profile

        #This will open NIR band and read its data
        with rasterio.open(nir_band_path) as src_nir:
            nir = src_nir.read(1).astype(float)

        # Ensure bands have the same shape - crucial for arithmetic operations
        if red.shape != nir.shape:
            # If shapes differ, crop to the smallest common extent
            min_rows = min(red.shape[0], nir.shape[0])
            min_cols = min(red.shape[1], nir.shape[1])
            red = red[:min_rows, :min_cols]
            nir = nir[:min_rows, :min_cols]
            print(f"Warning: Bands had different shapes. Reshaped to common size: {red.shape}")

        # --- Calculate NDVI ---
        # Handle zero or near-zero values to prevent division by zero errors
        # Set areas with 0 (often no-data or water) to NaN to exclude from NDVI
        red[red == 0] = np.nan
        nir[nir == 0] = np.nan

        # Perform NDVI calculation. Use np.errstate to suppress warnings for NaN operations.
        with np.errstate(divide='ignore', invalid='ignore'):
            ndvi = (nir - red) / (nir + red)

        # Set NoData value for areas where NDVI couldn't be calculated (NaNs)
        ndvi[np.isnan(ndvi)] = -9999 # A common no-data value in remote sensing

        print("NDVI calculated successfully!")

        # --- Prepare profile for saving the new NDVI GeoTIFF ---
        profile.update(
            driver='GTiff',
            dtype=rasterio.float32, # NDVI values are float, so use float32
            count=1, # NDVI is a single band image
            compress='lzw', # Apply LZW compression to reduce file size
            nodata=-9999 # Explicitly set the nodata value in the output file
        )
        # Update width and height in profile in case bands were reshaped
        profile.update(width=ndvi.shape[1], height=ndvi.shape[0])


        # --- Store and visualize this NDVI data ---
        # Construct an output filename based on the input band name
        # This extracts the unique part (like T30UWF_20250103T112451)
        try:
            # Assumes the filename structure: TILEID_DATETIME_BAND_RESOLUTION.jp2
            # e.g., T30UWF_20250103T112451_B04_10m.jp2
            # We want 'T30UWF_20250103T112451'
            filename_parts = os.path.basename(red_band_path).split('_')
            # Join the tile ID and datetime part
            filename_stem = f"{filename_parts[0]}_{filename_parts[1]}"
            output_ndvi_filename = f"ndvi_{filename_stem}.tif"
        except IndexError:
            # Fallback if the filename parsing doesn't match expected pattern
            output_ndvi_filename = "ndvi_calculated_image.tif"

        # Define the full path where the NDVI GeoTIFF will be saved (back to My Drive)
        output_ndvi_path = os.path.join(download_directory, output_ndvi_filename)

        # Write the NDVI array to a new GeoTIFF file
        with rasterio.open(output_ndvi_path, 'w', **profile) as dst:
            dst.write(ndvi.astype(rasterio.float32), 1)

        print(f"NDVI GeoTIFF saved to: {output_ndvi_path}")

        # --- Visualize the NDVI map using matplotlib and rasterio.plot ---
        plt.figure(figsize=(10, 10)) # Set figure size for better viewing
        with rasterio.open(output_ndvi_path) as src:
            # Display the raster. cmap='RdYlGn' is good for NDVI (Red-Yellow-Green)
            # vmin/vmax set the color scale range (-0.2 to 0.9 is typical for NDVI)
            show(src, cmap='RdYlGn', vmin=-0.2, vmax=0.9, title="NDVI for Newcastle")
            plt.colorbar(label='NDVI Value') # Add a color bar
        plt.show() # Display the plot

    except Exception as e:
        print(f"An error occurred during band processing or NDVI calculation: {e}")
else:
    print("ERROR: Could not find the B04_10m.jp2 and/or B08_10m.jp2 band files in your 'My Drive' root folder.")
    print(f"Please ensure they are directly in: {download_directory}")
    print("You can verify files in Colab using: !ls -l '/content/drive/My Drive/'")

Set download directory to: /content/drive/My Drive
Searching for band files...
Found Red band (B04): /content/drive/My Drive/T30UWF_20250103T112451_B04_10m.jp2
Found NIR band (B08): /content/drive/My Drive/T30UWF_20250103T112451_B08_10m.jp2
NDVI calculated successfully!
NDVI GeoTIFF saved to: /content/drive/My Drive/ndvi_T30UWF_20250103T112451.tif
